In [1]:
import re
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm

/tmp/ipykernel_18986/3037818073.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
e_topics = pd.read_csv("../topics/e_topic.csv.zip")
s_topics = pd.read_csv("../topics/s_topic.csv.zip")
g_topics = pd.read_csv("../topics/g_topic.csv.zip")
topics = pd.concat([e_topics, s_topics, g_topics])
topics.head()

,topic,word,type
0,Экология,аудит системы,1
1,Экология,взаимодействие,1
2,Экология,внедрение,1
3,Экология,воздействие,1
4,Экология,восстановительные мероприятия,1


In [3]:
label2idx = {topic: i for i, topic in enumerate(sorted(topics["topic"].unique()))}
idx2label = {i: topic for i, topic in enumerate(sorted(topics["topic"].unique()))}

In [4]:
idx2word = {i: word for i, word in enumerate(topics["word"].tolist())}
word2idx = defaultdict(list)

for i, word in enumerate(topics["word"].tolist()):
    word2idx[word].append(i)

In [5]:
regex = re.compile("(" + "(?!\w)|".join(topics["word"].unique()) + "(?!\w))")

In [6]:
topics.groupby("topic")["word"].agg(list)

topic
Антикоррупция                         [безработица, форма, среда, коррупция, округле...
Безопасность и охрана труда           [распределение, безопасность, охрана, содружес...
Безопасность продукта                 [задолженность, гибкость, функционально, безоп...
Биоразнообразие                       [береговой, водно-болотные угодия, восстановле...
Благотворительность                   [звание, стипендия, конкурс, учеба, музеиныи, ...
Вода                                  [бассейн реки, вода, водные ресурсы, вододефиц...
Воздух                                [CO, NO2, O3, SO2, азот, аммиак, бензин, выбро...
Дивиденды и акционеры                 [столица, учредить, своевременныи, распределен...
Инвестиции и капитальные вложения     [индия, исходныи, резервныи, прибыль, активист...
Инновации                             [резервныи, воспроизводство, инвестпроект, нов...
Климат                                [CH4, CO2, GHG Protocol, N2O, альтернативная, ...
Лидерство                 

In [7]:
topics_vectors = np.zeros((topics["topic"].nunique(), topics.shape[0]), dtype=int)

for topic, topic_words in topics.groupby("topic")["word"].agg(list).items():
    for word in topic_words:
        for idx in word2idx[word]:
            topics_vectors[label2idx[topic], idx] = 1

In [8]:
matrix = np.zeros(topics_vectors.shape)
matrix.shape

(29, 4414)

In [9]:
def fit(vectors, labels):
    for vector, label in zip(vectors, labels):
        for i in range(matrix.shape[0]):
            if i == label2idx[label]:
                matrix[i] += vector
            else:
                matrix[i] -= vector


def test(vectors, labels):
    total = 0
    for vector, label in zip(vectors, labels):
        res = np.argmax(vector @ matrix.T)
        total += label == idx2label[res]
        if label != idx2label[res]:
            print("expected", label, "result", idx2label[res])
    print("accuracy", total / len(labels))

In [10]:
fit(topics_vectors, list(sorted(topics["topic"].unique())))

In [11]:
test(topics_vectors, list(sorted(topics["topic"].unique())))

accuracy 1.0


In [12]:
df = pd.read_csv("../topics/paragraphs.csv.zip", usecols=["cleaned_text"])
df = df.dropna()
# df = df[df["cleaned_text"].apply(lambda x: len(x.split()) > 1)]
# df = df.reset_index(drop=True)
df.shape

(1963163, 1)

In [13]:
topics = []
for text in tqdm(df["cleaned_text"].tolist()):
    vector = np.zeros(matrix.shape[1])
    words = regex.findall(text)
    if len(words) == 0:
        topics.append(None)
        continue
    for word in words:
        vector[word2idx[word]] = 1
    res = np.argmax(vector @ matrix.T)
    topics.append(idx2label[res])

  0%|          | 0/1963163 [00:00<?, ?it/s]

In [14]:
pd.Series(topics).to_csv("topics.csv.zip")

In [6]:
topics = pd.read_csv("topics.csv.zip", index_col=0, names=["topics"], header=0)
topics.head()

,topics
0,Оплата труда
1,Трудовые отношения
2,Лидерство
3,Отношения с потребителями
4,Лидерство


In [8]:
df = pd.read_csv("../topics/paragraphs.csv.zip")
df = df.dropna()
df = df.join(topics)
df.head()

/tmp/ipykernel_18986/1670128334.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../topics/paragraphs.csv.zip")


,company,year,sector,report_type,paragraph,original_text,cleaned_text,Экология,Климат,Энергия,Воздух,Вода,Отходы,Биоразнообразие,Социально-культурное воздействи,max_score,max_topic,topics
1,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,4.0,Обращение председателя совета директоров.........,обращение председатель совет директор параметр...,0.260378,0.160128,0.0,0.0,0.0,0.0,0.0,0.0,0.260378,Экология,Трудовые отношения
3,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,7.0,и сервисных услуг. Мы практически завершили дл...,сервисный услуга мы практически завершить длит...,0.130189,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.130189,Экология,Отношения с потребителями
9,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,14.0,Наша компания входит в число российских предпр...,наш компания входить число российский предприя...,0.130189,0.092450,0.0,0.0,0.0,0.0,0.0,0.0,0.130189,Экология,Отчетность и прозрачность
11,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,16.0,Компания разрабатывает и успешно внедряет соци...,компания разрабатывать успешно внедрять социал...,0.000000,0.092450,0.0,0.0,0.0,0.0,0.0,0.0,0.092450,Климат,Безопасность и охрана труда
12,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,17.0,Российский союз промышленников и предпринимате...,российский союз промышленник предприниматель д...,0.000000,0.130744,0.0,0.0,0.0,0.0,0.0,0.0,0.130744,Климат,Отчетность и прозрачность


In [11]:
e_topics = pd.read_csv("../topics/e_topic.csv.zip")
s_topics = pd.read_csv("../topics/s_topic.csv.zip")
g_topics = pd.read_csv("../topics/g_topic.csv.zip")
topics = pd.concat([e_topics, s_topics, g_topics])
topics.head()

,topic,word,type
0,Экология,аудит системы,1
1,Экология,взаимодействие,1
2,Экология,внедрение,1
3,Экология,воздействие,1
4,Экология,восстановительные мероприятия,1


In [24]:
re_dict = {}
sheets_dict = {}
base_dict = {}
len_topics = defaultdict(dict)

for topic, row in topics.groupby("topic")[["word", "type"]].agg(list).iterrows():
    if not sheets_dict.get(topic, False):
        sheets_dict[topic] = {}
        base_dict[topic] = {}

    for word, type in zip(row["word"], row["type"]):
        sheets_dict[topic][word] = type
        base_dict[topic][word] = 0
        if not len_topics[topic].get(type, False):
            len_topics[topic][type] = 0
        len_topics[topic][type] += 1

    re_dict[topic] = re.compile("(" + "(?!\w)|".join(row["word"]) + "(?!\w))")

In [25]:
def get_score(words: dict[str, int], topic: str) -> int:
    count_words = {key: 0 for key in len_topics[topic]}
    for word in words:
        count_words[sheets_dict[topic][word]] += 1
    base_percent = count_words[1] / len_topics[topic][1]
    advanced_percent = (
        count_words[2] / len_topics[topic][2] if count_words.get(2, False) else 0
    )
    if base_percent < 0.20:
        return 1
    elif base_percent < 0.40:
        return 2
    elif base_percent < 0.60:
        return 3
    elif base_percent < 0.80:
        return 4
    elif base_percent <= 1 and advanced_percent == 0:
        return 5
    elif base_percent <= 1 and advanced_percent < 0.40:
        return 6
    elif base_percent <= 1 and advanced_percent < 0.80:
        return 7
    elif base_percent <= 1 and advanced_percent <= 1:
        return 7
    else:
        return 0

In [26]:
grouped = pd.DataFrame(
    df.groupby(["company", "year", "topics"])["cleaned_text"].agg(" ".join)
)
grouped.head()

cleaned_text
company year       topics                                                                              
Abbott  2013, 2012 Безопасность и охрана труда        о компания это один крупный многопрофильный ко...
                   Вода                               фото вверху контролировать водопотребление сво...
                   Инвестиции и капитальные вложения  долгосрочный цель мочь достигнуть систематичес...
                   Лидерство                          корпоративный этика стремиться соответствовать...
                   Отчетность и прозрачность          подразделение детский лечебный питание мы разр...

In [28]:
import copy

scores = []

for (company, year, topic), row in tqdm(grouped.iterrows(), total=grouped.shape[0]):
    paragraph_dict = copy.deepcopy(base_dict)[topic]
    text = row["cleaned_text"]
    words = re_dict[topic].findall(text)
    for word in words:
        paragraph_dict[word] = 1
    scores.append(get_score(paragraph_dict, topic))

  0%|          | 0/24352 [00:00<?, ?it/s]

In [29]:
grouped["score"] = scores
grouped.head(20)

cleaned_text  \
company     year       topics                                                                                  
Abbott      2013, 2012 Безопасность и охрана труда         о компания это один крупный многопрофильный ко...   
                       Вода                                фото вверху контролировать водопотребление сво...   
                       Инвестиции и капитальные вложения   долгосрочный цель мочь достигнуть систематичес...   
                       Лидерство                           корпоративный этика стремиться соответствовать...   
                       Отчетность и прозрачность           подразделение детский лечебный питание мы разр...   
                       Трудовые отношения                  содержание развитие система здравоохранение ге...   
                       Эффективность и производительность  корпоративный социальный ответственность росси...   
Alcoa, Inc. 2004       Безопасность и охрана труда         природосберегающий использоваться кроме упаков...   
                       Благотворительность                       рик белёд исполнительный европейский регион   
                       Воздух                                         окисел азот в тысяча метрический тонна   
                       Инвестиции и капитальные вложения   один проблема который постоянно сталкиваться я...   
                       Лидерство                           устойчивый развитие компания основание тот уси...   
                       Отходы                              работать заказчик компания разработать система...   
                       Отчетность и прозрачность           возрастание выброс происходить связь применени...   
                       Трудовые отношения                  наш преимущество заключаться производить очень...   
                       Управление рисками                  показатель экологический безопасность для комп...   
                       Экология                            паний также внести свой вклад снижение объём в...   
                       Эффективность и производительность  я очень приятно представить ваш внимание первы...   
            2006       Безопасность и охрана труда         му разочаровать результат свой работа предотвр...   
                       Благотворительность                 право человек в вопрос соблюдение право челове...   

                                                           score  
company     year       topics                                     
Abbott      2013, 2012 Безопасность и охрана труда             5  
                       Вода                                    7  
                       Инвестиции и капитальные вложения       5  
                       Лидерство                               5  
                       Отчетность и прозрачность               5  
                       Трудовые отношения                      5  
                       Эффективность и производительность      5  
Alcoa, Inc. 2004       Безопасность и охрана труда             5  
                       Благотворительность                     5  
                       Воздух                                  5  
                       Инвестиции и капитальные вложения       5  
                       Лидерство                               5  
                       Отходы                                  7  
                       Отчетность и прозрачность               5  
                       Трудовые отношения                      5  
                       Управление рисками                      5  
                       Экология                                7  
                       Эффективность и производительность      5  
            2006       Безопасность и охрана труда             5  
                       Благотворительность                     5

In [30]:
grouped.to_csv("grouped_paragraphs_expert_algo.csv.zip")